In [ ]:
import pypsa

In [ ]:
# n = pypsa.Network("../../results/validation_2023/networks/base_s_39__100seg_2025.nc")
n = pypsa.Network("../../resources/validation_2023/networks/base_s_39___2025.nc")

e_nom ("size of tank") = number_cars * bev_energy * bev_dsm_availability * land_transport_electric_share

In [ ]:
ev_battery = n.stores.query("carrier == 'EV battery'").index
n.stores.loc[ev_battery].head()

e_min_pu ("minimum per unit charge"), a periodic timeseries

In [ ]:
n.stores_t.e_min_pu.iloc[0:24*7][list(ev_battery[0:4])].plot()

multiplied with the "tank size", this is the resulting "minimum charge" - still a periodic timeseries

In [ ]:
((n.stores_t.e_min_pu.multiply(n.stores.e_nom)).iloc[0:24*7][list(ev_battery[0:4])]).plot()

exemplary demand for BEVs - timeseries

In [ ]:
ev_load = n.loads.query("carrier == 'land transport EV'").index
n.loads_t.p_set.multiply(n.snapshot_weightings.generators,axis=0)[list(ev_load[0:3])].iloc[0:24*5].plot()

p_nom ("how many MW can be charged within a single hour") = number_cars * bev_charge_rate * land_transport_electric_share

In [ ]:
bev_charge = n.links.query("carrier == 'BEV charger'").index
n.links.loc[bev_charge].head()

p_max_pu ("cannot charge more than this per unit value") = avail_max - (avail_max - avail_mean) * (traffic - traffic.min()) / (traffic.mean() - traffic.min())
> linear scaling, highest when traffic is lowest, decreases if traffic increases

In [ ]:
n.links_t.p_max_pu[bev_charge].iloc[0:24*7][list(bev_charge[0:4])].plot()

multiplied with the charging capacity (p_nom), this results in the "maximum MWh increase"

In [ ]:
n.links_t.p_max_pu.multiply(n.links.p_nom,axis=1)[bev_charge].iloc[0:24*7][list(bev_charge[0:4])].plot()

if v2g is enabled, an additional link is added that goes from the EV to the LV grid (we assume all EVs are charged at LV)

p_nom ("capacity") = number_cars * bev_charge_rate * electric_share * bev_dsm_availability

In [ ]:
v2g = n.links.query("carrier == 'V2G'").index
n.links.loc[v2g].head()

per unit availability is the same as for v2g (basically they can dms or v2g when plugged into the grid)

In [ ]:
n.links_t.p_max_pu[v2g].iloc[0:24*7][list(v2g[0:3])].plot()

absolute v2g contribution is lower than dsm though, depending on the "bev_dsm_availability" parameter

In [ ]:
n.links_t.p_max_pu.multiply(n.links.p_nom)[v2g].iloc[0:24*7][list(v2g[0:3])].plot()